In [1]:
from bs4 import BeautifulSoup as bs
import requests
import time
import sys
import pandas as pd
import numpy as np
import re
import datetime

In [2]:
Mois=['Janvier','Février','Mars','Avril','Mai','Juin','Juillet','Août','Septembre','Octobre','Novembre','Décembre']

In [3]:
def getPage(id):
    # get url
    url = "https://www.avito.ma/fr/maroc/voitures-%C3%A0_vendre?o="+str(id);
    # get page
    page = requests.get(url)
    # page form html
    container = bs(page.content,'html.parser')
    #find paragraphe using tagname and class
    container = container.find("div",{"class":"sc-1nre5ec-0 gpXkJn listing"})
    hrefs = [href.get("href") for href in container.findAll("a")]
    return hrefs

In [4]:
def getVilleAndDate(villeAndDate):    
    ville = villeAndDate[0].text
    date = villeAndDate[1].text
    if re.search("^..:..$", date):
        date = datetime.datetime.now()
        date = date.date().month
    else:
        date = date[3:]
        date = Mois.index(date)+1
    return ville, date

In [58]:
def getPrice(price):
    price = price.text
    if price == "Prix non specifié":
        print(price)
        price = price[0:-3]
    
    return price

In [84]:
def getDataOfCar(url):
    page = requests.get(url)
    container = bs(page.content,'html.parser')
    
    information = container.find("div",{"class":"sc-1g3sn3w-7 bOkEPW"})
    
    # get localisation et date
    villeAndDate = information.findAll("span",{"class":"sc-1x0vz2r-0 jWjZZH"})
    ville, date = getVilleAndDate(villeAndDate)
    print(ville)
    # get price 
    price = information.find('p',{"class":"sc-1x0vz2r-0 izYwIp"})
    if price:
        price = getPrice(price)
    else:
        price = np.nan
    print(price)
    carburant = information("span",{"class":"sc-1x0vz2r-0 eqWUao"})[0].get_text()
    puissanceFiscale = information("span",{"class":"sc-1x0vz2r-0 eqWUao"})[1].get_text()
    boiteVitesse = information("span",{"class":"sc-1x0vz2r-0 eqWUao"})[2].get_text()
    if carburant == '--':
        carburant = np.nan
    if puissanceFiscale == '--':
        puissanceFiscale = np.nan
    if boiteVitesse == '--':
        boiteVitesse = 'Manuelle'
    print(carburant)
    # the details
    details = container.find("div",{"class":"sc-1g3sn3w-8 gvDFlQ"}) 
    keys = details.find_all('span',{'class':'sc-1x0vz2r-0 gwdnQV'})
    values = details.find_all('span',{'class':'sc-1x0vz2r-0 hrMdWO'})
    detail = {
        'Marque': np.nan, 'Modèle': np.nan, 'Année-Modèle': np.nan, 'Kilométrage': np.nan,
        'Nombre de portes': np.nan, 'Origine': np.nan, 'Première main': np.nan, 
        'Type': np.nan, 'Secteur': np.nan,'État':np.nan
    }
    for key,value in zip(keys,values):
        detail[key.text] = value.text
    print(detail)
    l = list(detail.values())
    infos = list()
    infos.append(price)
    infos.append(ville)
    infos.append(date)
    infos.append(carburant)
    infos.append(puissanceFiscale)
    infos.append(boiteVitesse)
    infos = infos + l
    
    return infos

In [91]:
listFinal = list()
for i in range(1):
    listOfUrl = getPage(i)
    j = 0
    for url in listOfUrl:
        infos.append(url)
        print(i,' : ', j,' = ',url)
        j += 1
      
        info = getDataOfCar(url)
        info.append(url)
        listFinal.append(info)  
        time.sleep(1.2)

0  :  0  =  https://www.avito.ma/fr/casablanca/voitures/Clio_4_gt_Line_49059920.htm
Casablanca
132 000 DH
Diesel
{'Marque': 'Renault', 'Modèle': 'Clio', 'Année-Modèle': '2019', 'Kilométrage': '90 000 - 94 999', 'Nombre de portes': '5', 'Origine': 'WW au Maroc', 'Première main': nan, 'Type': "Voitures d'occasion, Vente", 'Secteur': nan, 'État': 'Excellent'}
0  :  1  =  https://www.avito.ma/fr/hay_el_qods/voitures/E220_D_Mercedes__47826158.htm
Oujda
nan
Diesel
{'Marque': 'Mercedes-Benz', 'Modèle': 'Classe E', 'Année-Modèle': '2021', 'Kilométrage': '0 - 4 999', 'Nombre de portes': '5', 'Origine': 'Importée neuve', 'Première main': 'Oui', 'Type': "Voitures d'occasion, Vente", 'Secteur': 'Hay El Qods', 'État': 'Excellent'}
0  :  2  =  https://www.avito.ma/fr/autre_secteur/voitures/Mitsubishi_Lancer_Toutes_options__49069422.htm
Meknès
53 000 DH
Essence
{'Marque': 'Mitsubishi', 'Modèle': 'lancer', 'Année-Modèle': '2007', 'Kilométrage': '190 000 - 199 999', 'Nombre de portes': '5', 'Origine': 

0  :  23  =  https://www.avito.ma/fr/oulfa/voitures/vente_grand_cherroke_1er_main_47012594.htm
Casablanca
200 000 DH
Diesel
{'Marque': 'Jeep', 'Modèle': 'Grand Cherokee', 'Année-Modèle': '2016', 'Kilométrage': '140 000 - 149 999', 'Nombre de portes': '5', 'Origine': 'WW au Maroc', 'Première main': 'Oui', 'Type': "Voitures d'occasion, Vente", 'Secteur': 'Oulfa', 'État': 'Excellent'}
0  :  24  =  https://www.avito.ma/fr/al_fida/voitures/Renault_Megane_Noir_20_49112873.htm
Casablanca
180 000 DH
Diesel
{'Marque': 'Renault', 'Modèle': 'Megane', 'Année-Modèle': '2020', 'Kilométrage': '25 000 - 29 999', 'Nombre de portes': '5', 'Origine': 'WW au Maroc', 'Première main': 'Oui', 'Type': "Voitures d'occasion, Vente", 'Secteur': 'Al Fida', 'État': 'Excellent'}
0  :  25  =  https://www.avito.ma/fr/salé/voitures/Hyundai_i30_Diesel_48638593.htm
Salé
nan
Diesel
{'Marque': 'Hyundai', 'Modèle': 'i 30', 'Année-Modèle': '2013', 'Kilométrage': '140 000 - 149 999', 'Nombre de portes': '5', 'Origine': 'WW a

In [92]:
sys.getsizeof(listFinal)

376

In [99]:
df = pd.DataFrame(listFinal, columns =[
                'price', 'ville', 'date', 'carburant', 'puissance fiscale', 'boite a vitesse', 'Marque', 'Modèle',
               'Année-Modèle', 'Kilométrage', 'Nombre de portes', 'Origine', 'Première main','Type','secteur','État','url'

            ])

df

,price,ville,date,carburant,puissance fiscale,boite a vitesse,Marque,Modèle,Année-Modèle,Kilométrage,Nombre de portes,Origine,Première main,Type,secteur,État,url
0,132 000 DH,Casablanca,3,Diesel,6 CV,Manuelle,Renault,Clio,2019,90 000 - 94 999,5,WW au Maroc,NaN,"Voitures d'occasion, Vente",NaN,Excellent,https://www.avito.ma/fr/casablanca/voitures/Cl...
1,NaN,Oujda,3,Diesel,10 CV,Automatique,Mercedes-Benz,Classe E,2021,0 - 4 999,5,Importée neuve,Oui,"Voitures d'occasion, Vente",Hay El Qods,Excellent,https://www.avito.ma/fr/hay_el_qods/voitures/E...
2,53 000 DH,Meknès,3,Essence,7 CV,Manuelle,Mitsubishi,lancer,2007,190 000 - 199 999,5,WW au Maroc,Non,"Voitures d'occasion, Vente",Autre secteur,Excellent,https://www.avito.ma/fr/autre_secteur/voitures...
3,NaN,Oujda,3,Hybride,8 CV,Automatique,Mercedes-Benz,Classe GLE,2021,0 - 4 999,5,Importée neuve,Oui,"Voitures d'occasion, Vente",Hay El Qods,NaN,https://www.avito.ma/fr/hay_el_qods/voitures/G...
4,NaN,Oujda,3,Essence,23 CV,Manuelle,Mercedes-Benz,Classe GLC,2020,20 000 - 24 999,NaN,NaN,NaN,"Voitures d'occasion, Vente",Hay El Qods,NaN,https://www.avito.ma/fr/hay_el_qods/voitures/M...
5,NaN,Oujda,3,Diesel,6 CV,Automatique,Cupra,Formentor,2021,0 - 4 999,5,Importée neuve,Oui,"Voitures d'occasion, Vente",Hay El Qods,Excellent,https://www.avito.ma/fr/hay_el_qods/voitures/C...
6,NaN,Casablanca,3,Diesel,8 CV,Manuelle,Audi,Q3,2016,95 000 - 99 999,5,Dédouanée,Oui,"Voitures d'occasion, Vente",Aïn Chock,Excellent,https://www.avito.ma/fr/ain_chock/voitures/Aud...
7,NaN,Oujda,3,Diesel,8 CV,Automatique,Mercedes-Benz,Classe C,2021,0 - 4 999,5,Importée neuve,Oui,"Voitures d'occasion, Vente",Hay El Qods,Excellent,https://www.avito.ma/fr/hay_el_qods/voitures/N...
8,NaN,Oujda,3,Diesel,8 CV,Automatique,Mercedes-Benz,Classe E,2021,0 - 4 999,3,Importée neuve,Oui,"Voitures d'occasion, Vente",Hay El Qods,Excellent,https://www.avito.ma/fr/hay_el_qods/voitures/E...
9,300 000 DH,Rabat,3,Diesel,6 CV,Automatique,Ford,Kuga,2021,15 000 - 19 999,5,NaN,NaN,"Voitures d'occasion, Vente",NaN,Excellent,https://www.avito.ma/fr/rabat/voitures/ford_ku...


In [100]:
df['url']

0     https://www.avito.ma/fr/casablanca/voitures/Cl...
1     https://www.avito.ma/fr/hay_el_qods/voitures/E...
2     https://www.avito.ma/fr/autre_secteur/voitures...
3     https://www.avito.ma/fr/hay_el_qods/voitures/G...
4     https://www.avito.ma/fr/hay_el_qods/voitures/M...
5     https://www.avito.ma/fr/hay_el_qods/voitures/C...
6     https://www.avito.ma/fr/ain_chock/voitures/Aud...
7     https://www.avito.ma/fr/hay_el_qods/voitures/N...
8     https://www.avito.ma/fr/hay_el_qods/voitures/E...
9     https://www.avito.ma/fr/rabat/voitures/ford_ku...
10    https://www.avito.ma/fr/hay_el_qods/voitures/A...
11    https://www.avito.ma/fr/ain_chock/voitures/Aud...
12    https://www.avito.ma/fr/ben_souda/voitures/Pas...
13    https://www.avito.ma/fr/casablanca/voitures/Re...
14    https://www.avito.ma/fr/taourirt/voitures/Daci...
15    https://www.avito.ma/fr/tanger/voitures/Kia_Ri...
16    https://www.avito.ma/fr/safi/voitures/Nissan_Q...
17    https://www.avito.ma/fr/ain_diab/voitures/

In [101]:
df.to_excel("output.xlsx")  